In [1]:
from autogen_ext.models.openai import OpenAIChatCompletionClient

custom_model_client = OpenAIChatCompletionClient(
    model="Qwen/Qwen3-0.6B",
    base_url="http://localhost:8000/",
    api_key="placeholder",
    model_info={
        "vision": False,
        "function_calling": True,
        "json_output": True,
        "family": "qwen",
        "structured_output": True,
    },
)

In [2]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat

# Create the primary agent.
primary_agent = AssistantAgent(
    name="primary",
    model_client=custom_model_client,
    system_message=(
        "You are a writing assistant."
        "If you are given feedback on a draft, use it to revise the draft. "
        "If only a prompt is given, write a new version accordingly."
        "Respond to the user without any prefacing or follow-up statements."
        "Do not write things like:"
        "    •	“Sure!”"
        "    •	“Of course!”"
        "    •	“Here is the result:”"
        "    •	“Let me know if you need more help.”"
        "Return only the direct answer or output, nothing else."
    )
)

# Create the critic agent.
critic_agent = AssistantAgent(
    "critic",
    model_client=custom_model_client,
    system_message="""\
        You are a very strict poetry critic.

        Behavior:
        1. Whenever you receive a text and **none** of your previous messages in this chat include detailed feedback (i.e. this is your first critique), you must give **only** detailed, constructive feedback.  Do **not** say APPROVE right away.
        2. If you receive a text and **your last message** in this conversation contained numbered feedback items, then treat this as a revision:
        3. If your feedback is addressed and the generated text seem good respond with exactly in all-caps: 'APPROVE'
        4. Otherwise, provide further feedback—do **not** say APPROVE.
        5. Never approve a first submission
        6. Do not compliment the text or repeat it whole, just the reference and the feedback.""",
)


# Define a termination condition that stops the task if the critic approves.
text_termination = TextMentionTermination("APPROVE")

# Create a team with the primary and critic agents.
team = RoundRobinGroupChat([primary_agent, critic_agent], termination_condition=text_termination, max_turns=3)

In [3]:
result = await team.run(task="Write a short poem about the fall season.")
print(result)

messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=125, completion_tokens=40), metadata={}, content='The wind howls through the trees,  \nThe earth turns in a slow, warm hue.  \nA cloak of frost wraps the sky,  \nAnd the world holds its breath, in peace.', type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=244, completion_tokens=4), metadata={}, content='APPROVE', type='TextMessage')] stop_reason="Text 'APPROVE' mentioned"


In [4]:
from autogen_agentchat.base import TaskResult

await team.reset()  # Reset the team for a new task.
async for message in team.run_stream(task="Write a short poem about the fall season."):  # type: ignore
    if isinstance(message, TaskResult):
        print("Stop Reason:", message.stop_reason)
    else:
        print(message)

source='user' models_usage=None metadata={} content='Write a short poem about the fall season.' type='TextMessage'
source='primary' models_usage=RequestUsage(prompt_tokens=125, completion_tokens=39) metadata={} content='The leaves turn to gold and crimson,  \nThe wind whispers secrets deep and true.  \nThe earth turns gold in the night’s embrace,  \nAnd the frost paints the world in white.' type='TextMessage'
source='critic' models_usage=RequestUsage(prompt_tokens=243, completion_tokens=4) metadata={} content='APPROVE' type='TextMessage'
Stop Reason: Text 'APPROVE' mentioned


In [5]:
from autogen_agentchat.ui import Console

await team.reset()  # Reset the team for a new task.
await Console(team.run_stream(task="Write a two-sentence story about a sci-fi cowboy."))  # Stream the messages to the console.

---------- TextMessage (user) ----------
Write a two-sentence story about a sci-fi cowboy.
---------- TextMessage (primary) ----------
A futuristic cowboy named Eli rode through neon cities, his silver armor gleaming as he fought against an alien threat.
---------- TextMessage (critic) ----------
A futuristic cowboy named Eli rode through neon cities, his silver armor gleaming as he fought against an alien threat, where time and space twisted into unpredictable patterns.
---------- TextMessage (primary) ----------
A futuristic cowboy named Eli rode through neon cities, his silver armor gleaming as he fought against an alien threat, where time and space twisted into unpredictable patterns.


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Write a two-sentence story about a sci-fi cowboy.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=128, completion_tokens=24), metadata={}, content='A futuristic cowboy named Eli rode through neon cities, his silver armor gleaming as he fought against an alien threat.', type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=231, completion_tokens=33), metadata={}, content='A futuristic cowboy named Eli rode through neon cities, his silver armor gleaming as he fought against an alien threat, where time and space twisted into unpredictable patterns.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=193, completion_tokens=33), metadata={}, content='A futuristic cowboy named Eli rode through neon cities, his silver armor gleaming as he fought against an alien threat, where time and 

In [6]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMessageTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console

# Create a tool for incrementing a number.
def increment_number(number: int) -> int:
    """Increment a number by 1."""
    return number + 1


# Create a tool agent that uses the increment_number function.
looped_assistant = AssistantAgent(
    "looped_assistant",
    model_client=custom_model_client,
    tools=[increment_number],  # Register the tool.
    system_message="You are a helpful AI assistant, call the tool to increment the number.",
)

# Termination condition that stops the task if the agent responds with a text message.
termination_condition = TextMessageTermination("looped_assistant")

# Create a team with the looped assistant agent and the termination condition.
team = RoundRobinGroupChat(
    [looped_assistant],
    termination_condition=termination_condition,
)

# Run the team with a task and print the messages to the console.
async for message in team.run_stream(task="Increment the number 5 to 10."):  # type: ignore
    print(type(message).__name__, message)

#await custom_model_client.close()


TextMessage source='user' models_usage=None metadata={} content='Increment the number 5 to 10.' type='TextMessage'
TextMessage source='looped_assistant' models_usage=RequestUsage(prompt_tokens=42, completion_tokens=18) metadata={} content='5 → 6 → 7 → 8 → 9 → 10' type='TextMessage'
TaskResult messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Increment the number 5 to 10.', type='TextMessage'), TextMessage(source='looped_assistant', models_usage=RequestUsage(prompt_tokens=42, completion_tokens=18), metadata={}, content='5 → 6 → 7 → 8 → 9 → 10', type='TextMessage')] stop_reason="Text message received from 'looped_assistant'"


In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import DiGraphBuilder, GraphFlow
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Create the writer agent
writer = AssistantAgent("writer", model_client=custom_model_client, system_message="Draft a short paragraph on climate change.")

# Create the reviewer agent
reviewer = AssistantAgent("reviewer", model_client=custom_model_client, system_message="Review the draft and suggest improvements.")

# Build the graph
builder = DiGraphBuilder()
builder.add_node(writer).add_node(reviewer)
builder.add_edge(writer, reviewer)

# Build and validate the graph
graph = builder.build()

# Create the flow
flow = GraphFlow([writer, reviewer], graph=graph)


In [8]:
await Console(flow.run_stream(task="Write a short paragraph about climate change."))

---------- TextMessage (user) ----------
Write a short paragraph about climate change.
---------- TextMessage (writer) ----------
Climate change is a global challenge that threatens ecosystems, human societies, and the planet's stability. Rising global temperatures, extreme weather events, and rising sea levels are disrupting natural systems and pushing populations toward vulnerable areas. Addressing climate change requires urgent action to reduce greenhouse gas emissions and protect biodiversity, ensuring a sustainable future for generations to come.
---------- TextMessage (reviewer) ----------
Climate change is a pressing global challenge that threatens ecosystems, human societies, and the planet's stability. Rising global temperatures, extreme weather events, and increasing sea levels are disrupting natural systems and pushing populations toward vulnerable areas. To address climate change, urgent action is needed to reduce greenhouse gas emissions and protect biodiversity, ensuring 

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Write a short paragraph about climate change.', type='TextMessage'), TextMessage(source='writer', models_usage=RequestUsage(prompt_tokens=33, completion_tokens=68), metadata={}, content="Climate change is a global challenge that threatens ecosystems, human societies, and the planet's stability. Rising global temperatures, extreme weather events, and rising sea levels are disrupting natural systems and pushing populations toward vulnerable areas. Addressing climate change requires urgent action to reduce greenhouse gas emissions and protect biodiversity, ensuring a sustainable future for generations to come.", type='TextMessage'), TextMessage(source='reviewer', models_usage=RequestUsage(prompt_tokens=104, completion_tokens=71), metadata={}, content="Climate change is a pressing global challenge that threatens ecosystems, human societies, and the planet's stability. Rising global temperatures, extrem

In [9]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import DiGraphBuilder, GraphFlow
from autogen_agentchat.ui import Console

# Create the writer agent
writer = AssistantAgent("writer", model_client=custom_model_client, system_message="Draft a short paragraph on climate change.")

# Create two editor agents
editor1 = AssistantAgent("editor1", model_client=custom_model_client, system_message="Edit the paragraph for grammar.")

editor2 = AssistantAgent("editor2", model_client=custom_model_client, system_message="Edit the paragraph for style.")

# Create the final reviewer agent
final_reviewer = AssistantAgent(
    "final_reviewer",
    model_client=custom_model_client,
    system_message="Consolidate the grammar and style edits into a final version.",
)

# Build the workflow graph
builder = DiGraphBuilder()
builder.add_node(writer).add_node(editor1).add_node(editor2).add_node(final_reviewer)

# Fan-out from writer to editor1 and editor2
builder.add_edge(writer, editor1)
builder.add_edge(writer, editor2)

# Fan-in both editors into final reviewer
builder.add_edge(editor1, final_reviewer)
builder.add_edge(editor2, final_reviewer)

# Build and validate the graph
graph = builder.build()

# Create the flow
flow = GraphFlow(
    participants=builder.get_participants(),
    graph=graph,
)

# Run the workflow
await Console(flow.run_stream(task="Write a short paragraph about climate change."))


---------- TextMessage (user) ----------
Write a short paragraph about climate change.
---------- TextMessage (writer) ----------
Climate change is a global challenge driven by human activities, including burning fossil fuels, deforestation, and industrial processes, which are increasing atmospheric greenhouse gas concentrations. These changes are leading to rising global temperatures, extreme weather events, and severe ecosystem disruptions, posing significant threats to both human well-being and the planet's sustainability.
---------- TextMessage (editor1) ----------
Climate change is a global challenge driven by human activities such as burning fossil fuels, deforestation, and industrial processes, which are increasing atmospheric greenhouse gas concentrations. These changes are causing rising global temperatures, extreme weather events, and severe ecosystem disruptions, posing significant threats to both human well-being and the planet's sustainability.
---------- TextMessage (edit

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Write a short paragraph about climate change.', type='TextMessage'), TextMessage(source='writer', models_usage=RequestUsage(prompt_tokens=33, completion_tokens=64), metadata={}, content="Climate change is a global challenge driven by human activities, including burning fossil fuels, deforestation, and industrial processes, which are increasing atmospheric greenhouse gas concentrations. These changes are leading to rising global temperatures, extreme weather events, and severe ecosystem disruptions, posing significant threats to both human well-being and the planet's sustainability.", type='TextMessage'), TextMessage(source='editor1', models_usage=RequestUsage(prompt_tokens=99, completion_tokens=63), metadata={}, content="Climate change is a global challenge driven by human activities such as burning fossil fuels, deforestation, and industrial processes, which are increasing atmospheric greenhouse g

In [10]:
from autogen_agentchat.agents import AssistantAgent, MessageFilterAgent, MessageFilterConfig, PerSourceFilter
from autogen_agentchat.teams import DiGraphBuilder, GraphFlow
from autogen_agentchat.ui import Console

# Create agents
researcher = AssistantAgent(
    "researcher", model_client=custom_model_client, system_message="Summarize key facts about climate change.", 
)
analyst = AssistantAgent("analyst", model_client=custom_model_client, system_message="Review the summary and suggest improvements.")
presenter = AssistantAgent(
    "presenter", model_client=custom_model_client, system_message="Prepare a presentation slide based on the final summary."
)

# Apply message filtering
filtered_analyst = MessageFilterAgent(
    name="analyst",
    wrapped_agent=analyst,
    filter=MessageFilterConfig(per_source=[PerSourceFilter(source="researcher", position="last", count=1)]),
)

filtered_presenter = MessageFilterAgent(
    name="presenter",
    wrapped_agent=presenter,
    filter=MessageFilterConfig(per_source=[PerSourceFilter(source="analyst", position="last", count=1)]),
)

# Build the flow
builder = DiGraphBuilder()
builder.add_node(researcher).add_node(filtered_analyst).add_node(filtered_presenter)
builder.add_edge(researcher, filtered_analyst).add_edge(filtered_analyst, filtered_presenter)

# Create the flow
flow = GraphFlow(
    participants=builder.get_participants(),
    graph=builder.build(),
)

# Run the flow
await Console(flow.run_stream(task="Summarize key facts about climate change."))


---------- TextMessage (user) ----------
Summarize key facts about climate change.


---------- TextMessage (researcher) ----------
Climate change is a long-term change in the Earth's climate system, primarily driven by human activities such as burning fossil fuels, deforestation, and industrial processes. Key facts include:

- **Global Warming**: The Earth's average temperature has risen, with the past 100 years showing a 1.1°C increase compared to pre-industrial times.
- **Melting Winters**: Sea levels rise, leading to more frequent and severe flooding and rising temperatures in polar regions.
- **Extreme Weather**: Increased frequency and intensity of hurricanes, droughts, heatwaves, and cold events are observed.
- **Ocean Acidification**: The oceans are acidifying, affecting marine life and ecosystems.
- **Greenhouse Gas Emissions**: Industrial and transportation emissions are the primary causes of climate change.
- **Consequences**: Impacts include loss of biodiversity, rising sea levels, and health risks from extreme weather.

These facts highlight the urgency of

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Summarize key facts about climate change.', type='TextMessage'), TextMessage(source='researcher', models_usage=RequestUsage(prompt_tokens=35, completion_tokens=201), metadata={}, content="Climate change is a long-term change in the Earth's climate system, primarily driven by human activities such as burning fossil fuels, deforestation, and industrial processes. Key facts include:\n\n- **Global Warming**: The Earth's average temperature has risen, with the past 100 years showing a 1.1°C increase compared to pre-industrial times.\n- **Melting Winters**: Sea levels rise, leading to more frequent and severe flooding and rising temperatures in polar regions.\n- **Extreme Weather**: Increased frequency and intensity of hurricanes, droughts, heatwaves, and cold events are observed.\n- **Ocean Acidification**: The oceans are acidifying, affecting marine life and ecosystems.\n- **Greenhouse Gas Emissions**:

In [11]:
from typing import Literal
from pydantic import BaseModel

class AgentResponse(BaseModel):
    thoughts: str
    response: Literal["happy", "sad", "neutral"]

In [12]:
function_calling_agent = AssistantAgent(
    name="function_calling_agent",
    model_client=custom_model_client,
    system_message="Categorize the input as happy, sad, or neutral following json format. Use provided tools.",
    tools=[AgentResponse],
)

In [13]:
await Console(function_calling_agent.run_stream(task="I wanna run in a field of daisies and eat candyy!!!"))


---------- TextMessage (user) ----------
I wanna run in a field of daisies and eat candyy!!!
---------- TextMessage (function_calling_agent) ----------
{
  "type": "happy"
}


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='I wanna run in a field of daisies and eat candyy!!!', type='TextMessage'), TextMessage(source='function_calling_agent', models_usage=RequestUsage(prompt_tokens=52, completion_tokens=10), metadata={}, content='{\n  "type": "happy"\n}', type='TextMessage')], stop_reason=None)